### SENTIMENT ANALYSIS

**Highlights:**
 <br> 1. A binary classification model
 <br> 2. Object Oriented Approach 
 <br> 3. AN end-point API that accepts english text and respond with the predicted sentiment 
 <br> 4. Early stopping and dropout to avoid model overfitting 
 
 Note: 
 <br> A. The Epochs for the deep learning models can be further enhanced in the range from 30-50 for very precise accuracy
 <br> B. For API validation data when there is quotes within the string data it must be a single quotes with-in double quotes, for example: "   'bad'  "

In [1]:
!pip3 install pandas
!pip install scikit-learn
!pip install nltk
!pip install fastapi
!pip install uvicorn

In [1]:
cd D:\\చదువు మరియు సర్టిఫికేట్లు\\చ-22\\చ-22-MachineLearning-NeuralNets Projects\\7. Sentiment Analysis API

D:\చదువు మరియు సర్టిఫికేట్లు\చ-22\చ-22-MachineLearning-NeuralNets Projects\7. Sentiment Analysis API


In [2]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import nltk 
nltk.download('punkt') 
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import re
import string
import pickle

from sklearn.model_selection import train_test_split
from sklearn import metrics

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from tensorflow.keras.layers import Flatten, LSTM, Dense, Embedding, Bidirectional
from tensorflow.keras.callbacks import EarlyStopping

from fastapi import FastAPI
import uvicorn 

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\vpara\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\vpara\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\vpara\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\vpara\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
pd.set_option('max_colwidth',6000)
data = pd.read_csv('airline_sentiment_analysis.csv',usecols = ['airline_sentiment','text'], low_memory = True)
data.airline_sentiment.replace('positive', 1, inplace=True)
data.airline_sentiment.replace('negative', 0, inplace=True)
data.head(3)

,airline_sentiment,text
0,1,@VirginAmerica plus you've added commercials to the experience... tacky.
1,0,"@VirginAmerica it's really aggressive to blast obnoxious ""entertainment"" in your guests' faces &amp; they have little recourse"
2,0,@VirginAmerica and it's a really big bad thing about it


In [318]:
# data.info()   # 11541 entries, 2 columns
# data.isna().sum() # no missing values

In [41]:
class sentimentanalysis:
    def __init__(self,data):
        self.data = data        
        
    def textpreprocessing(self,inp):
        
        Pattern1 = r"((http://)[^ ]*|(https://)[^ ]*|( www\.)[^ ]*)"
        Pattern2 = '@[^\s]+'
        stopword = set(stopwords.words('english'))
        inp = inp.lower()
        
        inp = re.sub(r"yrs", "years", inp)
        inp = re.sub(r"hrs", "hours", inp)
        inp = re.sub(r"bday", "b-day", inp)
        inp = re.sub(r"mother's", "mother", inp)
        inp = re.sub(r"mom's", "mom", inp)
        inp = re.sub(r"dad's", "dad", inp)
        inp = re.sub(r"hahah|hahaha|hahahaha", "haha", inp)
        
        inp = re.sub(r"can't", "can not", inp)
        inp = re.sub(r"wasn't", "was not", inp)
        inp = re.sub(r"don't", "do not", inp)
        inp = re.sub(r"aren't", "are not", inp)
        inp = re.sub(r"isn't", "is not", inp)
        inp = re.sub(r"won't", "will not", inp)
        inp = re.sub(r"shouldn't", "should not", inp)
        inp = re.sub(r"wouldn't", "would not", inp)       
        inp = re.sub(r"haven't", "have not", inp)
        inp = re.sub(r"hasn't", "has not", inp)        
        inp = re.sub(r"couldn't", "could not", inp)
        inp = re.sub(r"weren't", "were not", inp)
        inp = re.sub(r"didn't", "did not", inp)
        inp = re.sub(r"ain't", "am not", inp)
        inp = re.sub(r"haven't", "have not", inp)
        inp = re.sub(r"doesn't", "does not", inp)

        inp = re.sub(r"he's", "he is", inp)
        inp = re.sub(r"here's", "here is", inp)
        inp = re.sub(r"what's", "what is", inp)
        inp = re.sub(r"there's", "there is", inp)
        inp = re.sub(r"he's", "he is", inp)
        inp = re.sub(r"it's", "it is", inp)
        inp = re.sub(r"there's", "there is", inp)
        inp = re.sub(r"we're", "we are", inp)
        inp = re.sub(r"that's", "that is", inp)     
        inp = re.sub(r"who's", "who is", inp)
        inp = re.sub(r"that's", "that is", inp)
        inp = re.sub(r"where's", "where is", inp)
        inp = re.sub(r"what's", "what is", inp)
        
        inp = re.sub(r"they're", "they are", inp)
        inp = re.sub(r"you're", "you are", inp)
        inp = re.sub(r"i'm", "I am", inp)
        inp = re.sub(r"you've", "you have", inp)
        inp = re.sub(r"we're", "we are", inp)
        inp = re.sub(r"we've", "we have", inp)
        inp = re.sub(r"y'all", "you all", inp)
        inp = re.sub(r"would've", "would have", inp)
        inp = re.sub(r"it'll", "it will", inp)
        inp = re.sub(r"we'll", "we will", inp)
        inp = re.sub(r"he'll", "he will", inp)
        inp = re.sub(r"they'll", "they will", inp)
        inp = re.sub(r"they'd", "they would", inp) 
        inp = re.sub(r"they've", "they have", inp)
        inp = re.sub(r"i'd", "i would", inp)
        inp = re.sub(r"should've", "should have", inp)
        inp = re.sub(r"we'd", "we would", inp)
        inp = re.sub(r"i'll", "I will", inp)
        inp = re.sub(r"they're", "they are", inp)
        inp = re.sub(r"let's", "let us", inp)
        inp = re.sub(r"it's", "it is", inp)
        inp = re.sub(r"you're", "you are", inp)
        inp = re.sub(r"i've", "I have", inp)
        inp = re.sub(r"you'll", "you will", inp)
        inp = re.sub(r"you'd", "you would", inp)
        inp = re.sub(r"could've", "could have", inp)
        inp = re.sub(r"youve", "you have", inp)  

        inp = re.sub(Pattern1,'',inp)
        inp = re.sub(Pattern2,'', inp) 
        inp = inp.translate(str.maketrans("","",string.punctuation))
        
        
        tokens = word_tokenize(inp)
        my_tokens = [w for w in tokens if w not in stopword]
        wordLemm = WordNetLemmatizer()
        words=[]
        for w in my_tokens:
            if len(w)>1:
                ele = wordLemm.lemmatize(w)
                words.append(ele)

        return ' '.join(words)
    
    def model_data(self):
        self.data['text'] = self.data['text'].apply(lambda x: obj.textpreprocessing(x))
        return self.data
        
        
    def model_NB(self):            # Model 1 NAIVE BAYES
        self.model_data()
        self.count_vect =  CountVectorizer(max_features= 1000)
        self.feature_vector = self.count_vect.fit(self.data.text)
        self.data_features =  self.count_vect.transform(self.data.text)
        self.train_x_m1, self.test_x_m1, self.train_y_m1, self.test_y_m1 =  train_test_split(self.data_features, self.data.airline_sentiment, 
                                                     test_size = 0.3, random_state = 42)
        self.model_1 = MultinomialNB()
        self.model_1.fit(self.train_x_m1.toarray(), self.train_y_m1)
        self.predicted_model_1 = self.model_1.predict(self.test_x_m1.toarray())
        
        return self.count_vect, self.model_1
  
    def model_CNN(self):          # Model 2 Convolutional Neural Network
        self.model_data()
        self.text = self.data['text'].to_numpy()
        self.sentiment = self.data['airline_sentiment'].to_numpy()
        self.train_x_m2, self.test_x_m2, self.train_y_m2, self.test_y_m2  = train_test_split(self.text, self.sentiment, test_size=0.3,random_state = 42)
        self.vocab_size = 10000
        self.sequence_length = 1000
        self.embedding_dim = 16
        self.tokenizer = Tokenizer(num_words=self.vocab_size, oov_token="<OOV>")
        self.tokenizer.fit_on_texts(self.train_x_m2)
        self.train_sequences = self.tokenizer.texts_to_sequences(self.train_x_m2)        
        self.train_padded = pad_sequences(self.train_sequences, maxlen=self.sequence_length, padding='post', truncating='post')
        self.test_sequences = self.tokenizer.texts_to_sequences(self.test_x_m2)
        self.test_padded = pad_sequences(self.test_sequences, maxlen=self.sequence_length, padding='post', truncating='post')
        
        self.model_2 = Sequential()
        self.model_2.add(Embedding(self.vocab_size, self.embedding_dim, input_length=self.sequence_length))
        self.model_2.add(Conv1D(filters=128, kernel_size=5, activation='relu'))
        self.model_2.add(MaxPooling1D(pool_size=2))
        self.model_2.add(Flatten())
        self.model_2.add(Dense(1, activation='sigmoid'))
        self.model_2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
        self.callbacks = [EarlyStopping(patience=2)]
        self.history_model_2 = self.model_2.fit(self.train_padded, self.train_y_m2, epochs=3, validation_data=(self.test_padded, self.test_y_m2), callbacks=self.callbacks)

        return self.tokenizer, self.model_2
        
    def model_LSTM(self):             # Model 3 Recurrent Neural Network- LSTM

        lstm_out = 32
        self.model_3 = Sequential()        
        self.model_3.add(Embedding(self.vocab_size, self.embedding_dim, input_length=self.sequence_length))        
        self.model_3.add(Bidirectional(LSTM(lstm_out)))
        self.model_3.add(Dense(10, activation='relu'))
        self.model_3.add(Dense(1, activation='sigmoid'))
        self.model_3.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
        self.history_model_3 = self.model_3.fit(self.train_padded, self.train_y_m2, epochs=3, validation_data=(self.test_padded, self.test_y_m2), callbacks=self.callbacks)

        return self.model_3

    def __repr__(self):
        # All three model metrics
        self.model_NB()
        self.model_CNN()
        self.model_LSTM()

        model1_metrics = metrics.classification_report(self.test_y_m1, self.predicted_model_1)
        model2_metrics = pd.DataFrame(self.history_model_2.history)
        model3_metrics = pd.DataFrame(self.history_model_3.history)
        
        Output = ['THE METRICS FOR NAIVE BAYES CLASSIFIER: ', model1_metrics,'THE METRICS FOR CNN CLASSIFIER: ', str(model2_metrics), 'THE METRICS FOR LSTM CLASSIFIER: ',repr(model3_metrics)]

        return   '\n\n'.join(Output)

#### The Model Metrics 

In [42]:
obj = sentimentanalysis(data)
obj

253/253 [==============================] - 20s 75ms/step - loss: 0.4055 - accuracy: 0.8217 - val_loss: 0.2846 - val_accuracy: 0.8885


THE METRICS FOR NAIVE BAYES CLASSIFIER: 

              precision    recall  f1-score   support

           0       0.94      0.94      0.94      2771
           1       0.75      0.74      0.74       692

    accuracy                           0.90      3463
   macro avg       0.84      0.84      0.84      3463
weighted avg       0.90      0.90      0.90      3463


THE METRICS FOR CNN CLASSIFIER: 

       loss  accuracy  val_loss  val_accuracy
0  0.405491  0.821738  0.284574      0.888536

#### Sentiment Analysis API

In [43]:
obj = sentimentanalysis(data)
app = FastAPI(debug=True)
@app.get("/Sentiment Analysis")

def predict(Validation_Data: str):
    processed_data = obj.textpreprocessing(Validation_Data)
    
    x,y = obj.model_NB()
    trans_data_1 = x.transform([processed_data])    
    model_prediction_1 = y.predict(trans_data_1.toarray())
    
    df = pd.DataFrame({"input_data":[processed_data]})
    trans_data_2_3 = df["input_data"].to_numpy()
    
    m,n = obj.model_CNN()  
    trans_data_2_3_seq = m.texts_to_sequences(trans_data_2_3)
    trans_data_2_3_padded = pad_sequences(trans_data_2_3_seq, maxlen=1000, padding='post', truncating='post')
    model_predict_2 = n.predict(trans_data_2_3_padded)
    model_prediction_2 = 1 if model_predict_2[0][0] >= 0.70 else 0   

    a = obj.model_LSTM()
    model_predict_3 = a.predict(trans_data_2_3_padded)
    model_prediction_3 = 1 if model_predict_3[0][0] >= 0.70 else 0   
    
    output = [model_prediction_1,model_prediction_2,model_prediction_3]    
    output = ["Positive" if ele==1 else "Negative" for ele in output]
            
    api_out = {'The Review is:      '
               + '{}'.format(output[0]) + ' '+'as per Naive Bayes Classifier' + '  -->          ' 
               + '{}'.format(output[1]) + ' '+ 'as per CNN Classifier' + '  -->          ' 
               + '{}'.format(output[2]) + ' '+ 'as per LSTM Classifier' }
    
    return api_out
    

if __name__ == '__main__':
    uvicorn.run(app)


RuntimeError: This event loop is already running

INFO:     Started server process [18488]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:53156 - "GET / HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:53157 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:53157 - "GET /openapi.json HTTP/1.1" 200 OK
253/253 [==============================] - 142s 540ms/step - loss: 0.4513 - accuracy: 0.8065 - val_loss: 0.3050 - val_accuracy: 0.8776


# THE END